In [1]:
import fsspec
import xarray as xr

In [2]:
fs = fsspec.filesystem('s3', profile='esip-qhub')

In [4]:
url = 's3://esip-qhub/usgs/testing/1979_gm_ppt_2020_07_14.nc'

In [7]:
%%time
ds = xr.open_dataset(fs.open(url))

ValueError: b'CDF\x01\x00\x00\x00\x00' is not the signature of a valid netCDF4 file

In [2]:
# kubernetes units
# E,P,T,G,M,K   10^3, 10^6, ... bytes
# Ei, Pi, Ti, Gi, Mi, Ki = 2^10, 2^20, ... bytes

# https://cloud.google.com/kubernetes-engine/docs/concepts/cluster-architecture#node_allocatable

import math

def available_cpu(cpu : int):
    # 6% of the first core
    reserved = 0.06
    if cpu > 1:
       reserved = reserved + 0.01
    if cpu > 2:
       reserved = reserved + 0.005
    if cpu > 3:
       reserved = reserved + 0.005
    if cpu > 4:
       reserved = reserved + 0.0025 * (cpu - 4)
    return cpu - reserved

def available_memory(memory : int):
    # memory in bytes
    Gi = 2**30
    Mi = 2**20
    Ki = 2**10

    # using linux kernel requirements
    memory = memory * 0.9833 - 1.205e8

    eviction_threshold = 100*Mi

    if memory < Gi:
        return memory - (255 * Mi)

    reserved = min(memory, 4*Gi) * 0.25
    if memory > 4*Gi:
        reserved = reserved + min(memory - 4*Gi, 4*Gi) * 0.20
    if memory > 8*Gi:
        reserved = reserved + min(memory - 8*Gi, 8*Gi) * 0.10
    if memory > 16*Gi:
        reserved = reserved + min(memory - 16*Gi, 112*Gi) * 0.06
    if memory > 128*Gi:
        reserved = reserved + (memory - 128*Gi) * 0.02

    return memory - reserved - eviction_threshold

print(available_cpu(4))
print(16 * 2**30 / 1024 - 16399092)
print(available_memory(16 * 2**30) / 1024 - 13610740)

print(available_cpu(8))
print(32 * 2**30 / 1024 - 32888740)
print(available_memory(32 * 2**30) / 1024 - 29093796)

# so within the 20Mi for the estimate

3.92
378124.0
-20291.359605001286
7.91
665692.0
-12047.26791100204


In [3]:
available_cpu(4)

3.92

In [6]:
available_memory(32*2**30)/(2**30)

27.734516841973303